<a href="https://colab.research.google.com/github/kplr-training/Airflow/blob/main/Ateliers/Solutions/07-Dynamic%20Task%20Mapping%20-%20Params.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic Task Mapping : Gestion de fichiers avec l’utilisation de Paramètres Constants et Simples 

![Diagramme sans nom-Page-7 drawio](https://user-images.githubusercontent.com/123757632/231465882-a97ab30c-8f97-4e7d-8ad8-138dff9db310.png)

1 . Définir le DAG (Directed Acyclic Graph) Airflow qui sera planifiée pour s'exécuter quotidiennement à partir d'une date  : 

Création du DAG nommée "const_plus_single_param" qui est planifiée pour s'exécuter quotidiennement à partir du 8 avril 2023.

In [ ]:
from airflow import DAG
from airflow.decorators import task
from airflow.operators.bash import BashOperator
import random
from datetime import datetime

with DAG('const_plus_single_param', start_date=datetime(2023, 4, 8), schedule_interval='@daily', catchup=False) as dag:

2 . Le DAG contient deux tâches définies en utilisant le décorateur "@task" qui permet de les traiter comme des tâches indépendantes qui peuvent être exécutées en parallèle :

In [ ]:
@task
def generate_files():
  return [f"file_{nb}" for nb in range(random.randint(2, 5))]

La première tâche nommée generate_files est créée en utilisant le décorateur task. Cette tâche génère une liste de noms de fichiers aléatoires compris entre 2 et 5. La liste de noms de fichiers est retournée.

In [ ]:
@task
def download_file(path: str, file: str):
  return (f"echo '{path}/{file}' >> /tmp/out-const.txt")

La deuxième tâche nommée download_file est créée en utilisant le décorateur task. Cette tâche prend en paramètre un chemin de fichier et un nom de fichier. La tâche ajoute le chemin complet du fichier généré à un fichier de sortie nommé "/tmp/out-const.txt".

In [ ]:
print_file = BashOperator.partial(task_id="print_file", do_xcom_push=False).expand(
bash_command=download_file.partial(path="files/partner").expand(file=generate_files()))

La tâche print_file est créée en utilisant l'opérateur BashOperator.partial et task_id est défini sur "print_file". do_xcom_push=False est également spécifié pour éviter que la tâche précédente ne stocke des données dans XCom. Le bash_command est créé en utilisant download_file.partial pour spécifier le chemin et generate_files pour spécifier le nom du fichier. expand() est utilisé pour évaluer dynamiquement les arguments de la commande Bash à l'exécution.

In [ ]:
# Define the task that consumes the dataset
check_files = BashOperator(
  task_id="check_files",
  bash_command="cat /tmp/out-const.txt",
  retries=3,
)

La tâche check_files est créée en utilisant l'opérateur BashOperator. task_id est défini sur "check_files" et la commande Bash cat /tmp/out-const.txt est utilisée pour afficher le contenu du fichier de sortie.

In [ ]:
print_file >> check_files

Enfin, print_file >> check_files est utilisé pour spécifier que la tâche print_file doit être exécutée avant la tâche check_files dans le DAG.

![image](https://user-images.githubusercontent.com/123757632/231478409-d5506dae-1724-476b-a2a8-b035fa19ebd2.png)

Execution du Dag :

![image](https://user-images.githubusercontent.com/123757632/231479677-3a312274-72aa-4443-9f7b-349e59f25fbd.png)

Verification du log : 

![image](https://user-images.githubusercontent.com/123757632/231479922-264cd436-6694-49fe-9874-537e19da20ff.png)